# HW5 ENSEMBLING, TUNNING

# Импорт библиотек

In [ ]:
from IPython.display import clear_output
!pip install lightgbm==3.0
!pip install gdown
!pip install optuna
!pip install catboost
!pip install pytorch-tabnet
!pip install -U imbalanced-learn
clear_output()

In [2]:
import catboost
import itertools
import imblearn
import lightgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import scipy
import seaborn as sns
import sklearn
import sklearn.datasets
import xgboost

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score, average_precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import matthews_corrcoef
from pytorch_tabnet.tab_model import  TabNetRegressor

Функция для визуализации результата (можете модифицировать).


In [3]:
def plot_regression_predictions(tree_reg, x, y, max_depth, x_range = [0, 1],
                                y_range = [-1, 1]):
    """
    Visualize data and model predictions
    :param tree_reg: trained model,
    :param x: features
    :param y: true values
    :param max_depth: max tree depth
    :param x_range: x range
    :param y_range: y range
    """
    x_pred = np.linspace(x_range[0], x_range[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x_pred)

    plt.xlim(x_range)
    plt.ylim(y_range)

    plt.xlabel("$x$", fontsize=18)
    plt.ylabel("$y$", fontsize=18, rotation=0)

    plt.plot(x, y, "b.", label="data")
    plt.plot(x_pred, y_pred, "r.-", linewidth=2, label=f"max_depth = {max_depth}")
    plt.legend()

Продемонстрируйте результат [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) при изменении глубины дерева (`random_state=42`). Попробуйте значения глубины $2, 5, 6, 9, 12, 15$
Используйте для визуализации те же данные, на которых происходило обучение.


Примечание: Для построения серии картинок используйте `plt.subplot`.
Пример использования: [ссылка](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/shared_axis_demo.html#sphx-glr-gallery-subplots-axes-and-figures-shared-axis-demo-py).

# Задание 1. Bootstrap

В этом задании используйте датасет breast_cancer &mdash; классический датасет для задачи бинарной классификации. Обучите модели:

 - `DecisionTreeClassifier`
 -`RandomForestClassifier`
 -`LigthGBMClassifier`
 -`SVC`
 -`BaggingClassifier` с базовым класификатором - SVC .

Параметры моделей можете оставить по умолчанию или задать сами.

Для каждой модели посчитайте [корреляцию Мэтьюса](https://en.wikipedia.org/wiki/Phi_coefficient) &mdash; метрику для оценки качества бинарной классификации, в частности, устойчивую к дисбалансу классов, ([`sklearn.metrics.matthews_corrcoef`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html), подробнее почитать про его пользу можно [здесь](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7)) для предсказанного ею класса и реального

С помощью bootstrap-подхода постройте 90% доверительные интервалы для качества полученных моделей. Используйте функцию `bootstrap_metric` из лекции ().

Постройте [боксплоты](https://seaborn.pydata.org/generated/seaborn.boxplot.html) для качества полученных моделей

In [4]:
breast_cancer = sklearn.datasets.load_breast_cancer()
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [ ]:
x = breast_cancer.data
y = breast_cancer.target
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
# Your code here

Сделайте вывод о том, какие модели работают лучше.

**Напишите вывод**

## Формат результата

График с демонстрацией корреляции Мэтьюса для следующих моделей:

 - `DecisionTreeClassifier`
 -`RandomForestClassifier`
 -`LigthGBMClassifier`
 -`SVC`
 -`BaggingClassifier` с базовым класификатором - SVC .


Пример графика:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/Exercises/EX03/result_2_task_ex03.png" width="600">

# Задание 2. Обучение и подбор параметров

Загрузим датасет с рецептами (состав блюд и дополнительные данные о них) и рейтинге рецептов.

In [6]:
recipies = pd.read_csv("data/recipes.csv")
print(recipies.iloc[:, 0:5].head())

   calories  protein   fat  sodium  cakeweek
0     426.0     30.0   7.0   559.0       0.0
1     403.0     18.0  23.0  1439.0       0.0
2     165.0      6.0   7.0   165.0       0.0
3     547.0     20.0  32.0   452.0       0.0
4     948.0     19.0  79.0  1042.0       0.0


In [ ]:
recipies = recipies.loc[pd.isna(recipies).sum(axis=1)==0, :] # remove na
y = recipies['rating']
x = recipies.drop(["rating", "title"], axis=1)
x_train_all, x_test, y_train_all, y_test = train_test_split(x.values, y.values, train_size=0.7, random_state=42)

Постройте модель, предсказывающую рейтинг рецепта по всем имеющимся признакам. Для анализа качества моделей используйте метрику `MSE`.

Выберите 2 модели &mdash; один случайный лес и один бустинг из приведенных ниже.

1. `xgboost.XGBRegressor`
2. `xgboost.XGBRFRegressor` (случайный лес от xgboost)
3. `lightgbm.LGBMRegressor`
4. `lightgbm.LGBMRegressor(boosting_type="rf")` (случайный лес от lightgbm)
5. `catboost.CatBoostRegressor`

Кроме этого, используйте так же
`TabNetRegressor`. Установите число эпох (`max_epochs`, параметр функции `fit`) равным 30.

Для первых двух моделей попытайтесь оптимизировать их параметры, чтобы получить хорошее качество.

Сравните качество полученных моделей.

In [ ]:
# Your code here

Сравните качество моделей с гиперпараметрами, подобранными вручную и бустинга `LightGBM` с гиперпараметрами, оптимизированными автоматически с помощью фреймворка [`optuna`](https://optuna.org/).

`optuna` используется для подбора гиперпараметров различных моделей машинного обучения и встретится в этом курсе ещё неоднократно. В частности для оптимизации градиентного бустинга `LightGBM` используется функция `optuna.integration.lightgbm.LightGBMTunerCV`. Другие функции можно найти по [ссылке](https://optuna.readthedocs.io/en/stable/reference/integration.html).

Используйте параметры, полученные при оптимизации модели с помошью `optuna.integration.lightgbm.LightGBMTunerCV` для обучения модели `LightGBM`.

Сравните качество полученных моделей.

In [ ]:
?optuna.integration.lightgbm.LightGBMTunerCV

In [ ]:
dtrain = optuna.integration.lightgbm.Dataset(x_train_all, label=y_train_all)

params = {
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt"}

tuner = optuna.integration.lightgbm.LightGBMTunerCV(params=params,
                                                    train_set=dtrain,
                                                    verbose_eval=False,
                                                    early_stopping_rounds=100,
                                                    time_budget=600,
                                                    folds=KFold(n_splits=4, shuffle=True),
                                                    seed=42)

tuner.run()

print("\nBest score:", tuner.best_score)
best_params = tuner.best_params
print("\nBest parameters: ")
for key, value in best_params.items():
    print(f"{key}: {value}")

In [ ]:
# Your code here

## Формат результата

Получить значения MSE для моделей:

1. Случайный лес ~ 1.4
2. Бустинг меньше 1.45
3. TabNet ~ 1.5
4. С использованием optuna ~ 1.3

# Задание 3. Дисбаланс классов

Важно обращать внимание на сбалансированность классов в наборе.
Предположим, у нас есть некоторый набор данных со следующими метками классов:

In [ ]:
real_labels  = [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]

В наборе 16 объектов относятся к классу 0, 5 к классу 1.

Мы обучили две модели. Первая всегда выдает 0:

In [ ]:
model1_res = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Вторая сумела обнаружить некоторую закономерность в признаках:

In [ ]:
model2_res = [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1]

Рассчитаем точность Accuracy для этих моделей:

In [ ]:
print("Accuracy for model1: ", accuracy_score(real_labels, model1_res))
print("Accuracy for model2: ", accuracy_score(real_labels, model2_res))

Accuracy нельзя использовать, если данные не сбалансированы!!!. Для несбалансированных данных необходимо использовать свои метрики и модели. Одной из таких метрик является balanced accuracy. При вычислении данной метрики считается полнота (recall) отдельно для каждого класса и вычисляется среднее значение:

In [ ]:
# Balanced accuracy for model1 = (16/16+0/5)/2 = 0.5
print("Balanced accuracy for model1: ", balanced_accuracy_score(real_labels, model1_res))
# Balanced accuracy for model2 = (12/16+4/5)/2 = 0.775
print("Balanced accuracy for model2: ", balanced_accuracy_score(real_labels, model2_res))

**Всегда проверяйте** являются ли ваши данные сбалансированными и могут ли выбранные для оценки модели метрики работать с несбалансированными классами.

Загрузим датасет с различными биомаркерами пациентов с меланомой (обезличенный, информации о пациентах нет) и переменной, содержащей 1, если пациент ответил на иммунотерапию (терапия помогла пациенту и произошло уменьшение размеров опухоли) и 0, если не ответил. Количество пациентов, отвечающих на терапию сильно меньше пациентов, которым терапия не помогает, поэтому предсказание ответа пациента на терапию на основании биомаркеров &mdash; актуальная задача в онкологии. В данном задании вам предстоит попробовать её решить.

In [7]:
cancer = pd.read_table("data/Cancer_dataset_2.tsv", index_col='sample_id')
display(cancer.head())

# split the data on features (x) and dependant variable (y)
y = cancer['Response']
x = cancer.drop('Response', axis=1)
print('\nNumber of patients responded to immunotherapy:')
display(y.value_counts())

,IgG1/IgA,IL21,CXCL9,CXCL10,CD8A,GZMB,KLRC2,KLRC3,KLRC4,GNLY,TGFB1,Response
sample_id,,,,,,,,,,,,
SAM4b0175e8db6e,3.242746,0.001280,-0.002986,-0.036366,0.096658,0.063467,0.502058,-0.083862,0.053659,0.091930,61.934119,0
SAMd215b503f99a,2.139016,-0.000089,0.030495,0.243958,0.161128,0.565798,-0.203495,-0.026902,-0.035405,0.030125,103.265837,0
SAM7fb6987514a4,12.614972,0.008103,0.502043,0.530783,0.388455,0.528142,-0.156209,0.001147,-0.028690,0.260703,53.552817,0
SAMd636e3461955,6.365973,-0.000139,0.024035,0.115127,0.084455,0.200038,-0.387373,-0.057837,0.045938,0.073192,80.837318,0
SAMc0da5d48686d,2.764089,0.006107,0.015533,0.135470,0.067686,0.053499,-0.116040,0.063714,0.088201,0.082940,114.422926,0



Number of patients responded to immunotherapy:


0    228
1     37
Name: Response, dtype: int64

В данном случае имеет место несбалансированность классов в наборе данных: пациентов, ответивших на терапию, гораздо меньше.

Есть два способа работы с несбалансированными по классам данными. Первый способ &mdash; это получение стратифицированных выборок. Необходимо иметь одинаковую долю образцов каждого класса в тренировочной и тестовой выборках, иначе возникает риск получения смещённых выборок, что приводит к некорректной оценке качества модели. Второй способ &mdash; это использование специальных алгоритмов, учитывающих несбалансированность классов.


В данном задании вам нужно продемонстрировать эффективность различных подходов  работы с несбалансированными выборками. Для этого вы будете использовать три модели, представленные ниже:

1. [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), библиотека sklearn
2. [`RandomForestClassifier` с балансировкой классов](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), библиотека sklearn  - меняет стандартный вес каждого класса, равный 1, на долю класса во входных данных (см. `class_weight`).
3. [`BalancedRandomForestClassifier`](https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html), библиотека imblearn - сэмплирует псевдовыборки таким образом, что в каждой псевдовыборке, которая подается на вход модели, баланс классов оказывается "выправлен".

Оцените эффективность подходов с помощью кросс-валидации, производя разбиение с учетом репрезентации классов и без него. В качестве метрики, отображающей эффективность модели, используйте значения `accuracy` и `balanced_accuracy`. Проинтерпретируйте результаты.

In [ ]:
?imblearn.ensemble.BalancedRandomForestClassifier

In [ ]:
?cross_validate

Объекты, принадлежащие разным классам, распределены неравномерно. Для адекватной работы cross_validate нужно перемешать данные. Для этого используйте флаг shuffle=True, применяя `KFold` и `StratifiedKFold` (см. параметр `cv` в функции cross_validate).

In [ ]:
# Your code here

Какая модель лучше справляется с дисбалансом классов?

**Напишите вывод**

## Формат результата

Для моделей:
1. `RandomForestClassifier`, библиотека sklearn
2. `RandomForestClassifier с балансировкой классов`, библиотека sklearn
3. `BalancedRandomForestClassifier`, библиотека imblearn

Получить значения `accuracy` и `balanced_accuracy` при `KFold`-кроссвалидации и `StratifiedKFold`-кроссвалидации.

# Задание 4. Корреляция базовых моделей (maybe HARD - EXTRA)*

Для случайного леса, на примере датасета breast_cancer, постройте зависимость между попарными корреляциями базовых моделей и числом фичей, которые отбираются в каждую модель.


In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()

x = breast_cancer.data
y = breast_cancer.target

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
# Your code here

В параграфе лекции **Метод случайных подпространств** дана информация о рекомендованном количестве признаков для задачи классификации. Как соотносится полученный график и рекомендуемое количество признаков?

**Напишите вывод**

## Формат результата

График зависимости между попарными корреляциями базовых моделей и числом фичей, которые отбираются в каждую модель.

Пример графика:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/Exercises/EX03/result_5_task_ex03.png" width="600">